In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
data = pd.read_csv(r'train.csv',index_col=0,delimiter=';')
data.head(3)

,АвтомобильМодель,АвтомобильМодельБренд,АвтомобильVIN,Значение,АвтомобильГодВыпуска,Период,АвтомобильМодельМежсервисныйПробегТО,ТО
0,LADA Largus Caravan (7м),LADA,27837,29 644,01.01.2014 0:00:00,03.08.2015 0:00:00,15 000,Нет
1,LADA Largus Caravan (7м),LADA,27837,29 644,01.01.2014 0:00:00,06.08.2015 0:00:00,15 000,Да
2,LADA Largus Caravan (7м),LADA,27837,62 152,01.01.2014 0:00:00,28.04.2016 0:00:00,15 000,Нет


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 109236 entries, 0 to 109235
Data columns (total 8 columns):
 #   Column                                Non-Null Count   Dtype 
---  ------                                --------------   ----- 
 0   АвтомобильМодель                      109236 non-null  object
 1   АвтомобильМодельБренд                 109236 non-null  object
 2   АвтомобильVIN                         109236 non-null  int64 
 3   Значение                              108832 non-null  object
 4   АвтомобильГодВыпуска                  109236 non-null  object
 5   Период                                109236 non-null  object
 6   АвтомобильМодельМежсервисныйПробегТО  109236 non-null  object
 7   ТО                                    109236 non-null  object
dtypes: int64(1), object(7)
memory usage: 7.5+ MB


In [4]:
def parse_value(value: str):
    value_str = str(value)
    if value_str == 'nan':
        return value
    return int(''.join(value_str.split()))

def parse_date(date_str: str):
    day, month, year = date_str.split()[0].split('.')
    return int((float(year)-2000)*12 + float(month) + float(day)/31)

data['Значение'] = data['Значение'].map(lambda x: parse_value(x))
data['АвтомобильМодельМежсервисныйПробегТО'] = data['АвтомобильМодельМежсервисныйПробегТО'].map(lambda x: parse_value(x))
data['time_key'] = data['Период'].map(lambda x: parse_date(x))

In [28]:
import plotly.express as plyex
import statsmodels.api  as sm

In [189]:
%matplotlib qt

In [6]:
vins = data[data['АвтомобильМодель'] =='Toyota Land Cruiser Prado 150']['АвтомобильVIN'].unique()

In [25]:
df = data[data['АвтомобильМодель'] =='Toyota Land Cruiser Prado 150']
new_df = []
for _, group in data.groupby(by='АвтомобильVIN'):
    new_group  =group.sort_values(by='time_key')
    model = sm.OLS(new_group['Значение'],new_group['time_key'])
    res = model.fit()
    out = res.params
    new_df.append()

new_df = pd.concat(new_df)

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [27]:
a= new_df1[-1]
a

,АвтомобильМодель,АвтомобильМодельБренд,АвтомобильVIN,Значение,АвтомобильГодВыпуска,Период,АвтомобильМодельМежсервисныйПробегТО,ТО,time_key
16736,Toyota Camry VII,TOYOTA,46313,2346.0,01.01.2015 0:00:00,12.11.2015 0:00:00,10000,Нет,191
71407,Toyota Camry VII,TOYOTA,46313,9476.0,01.01.2015 0:00:00,14.08.2016 0:00:00,10000,Да,200
16737,Toyota Camry VII,TOYOTA,46313,19411.0,01.01.2015 0:00:00,13.08.2017 0:00:00,10000,Да,212
44058,Toyota Camry VII,TOYOTA,46313,29370.0,01.01.2015 0:00:00,03.09.2018 0:00:00,10000,Да,225
71408,Toyota Camry VII,TOYOTA,46313,32280.0,01.01.2015 0:00:00,21.11.2018 0:00:00,10000,Нет,227


In [197]:
for vin in vins:
    res = data[data['АвтомобильVIN'] ==vin  ]
    res1 = res.sort_values(by='time_key')
    plt.plot(
        np.array(res1['time_key']),
        np.array(res1['Значение']),
    )
    # plt.plot(
    #     np.array(res1['time_key'][res1['ТО'] == 'Да']),
    #     np.array(res1['Значение'][res1['ТО'] == 'Да']),
    #     # c='r'
    # )

In [24]:
plyex.line(
    new_df,
    x='time_key',
    y='Значение',
    color='АвтомобильМодельБренд',
    width=1500,
    height=800,
)